In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import logging
logging.basicConfig(level=logging.WARN)

![image](https://github.com/eWaterCycle/ewatercycle/raw/main/docs/examples/logo.png)

# User guide

This user manual will explain how the eWaterCycle Python package can be used to perform hydrological experiments. We will walk through the following chapters:

- parameter sets
- forcing data
- model instances
- using observations
- analysis

Each of these chapters correspond to a so-called "subpackage" of eWaterCycle Python package. Before we continue, however, we will briefly explain the configuration file.

### Configuration

To be able to find all needed data and models eWaterCycle comes with a configuration object. This configuration contains system settings for eWaterCycle (which container technology to use, where is the data located, etc). In general these should not need to be changed by the user for a specific experiment, and ideally a user would never need to touch this configuration on a properly managed system. However, it is good to know that it is there. 

You can see the default configuration on your system like so:

In [2]:
from ewatercycle import CFG
CFG

Config({'container_engine': 'singularity',
        'esmvaltool_config': None,
        'ewatercycle_config': PosixPath('/home/peter/.config/ewatercycle/ewatercycle.yaml'),
        'grdc_location': None,
        'output_dir': PosixPath('/home/peter/ewatercycle/ewatercycle/docs/examples'),
        'parameter_sets': {'lisflood_fraser': {'config': 'lisflood_fraser/settings_lat_lon-Run.xml',
                                               'directory': 'lisflood_fraser',
                                               'doi': 'N/A',
                                               'supported_model_versions': {'20.10'},
                                               'target_model': 'lisflood'},
                           'pcrglobwb_rhinemeuse_30min': {'config': 'pcrglobwb_rhinemeuse_30min/setup_natural_test.ini',
                                                          'directory': 'pcrglobwb_rhinemeuse_30min',
                                                          'doi': 'N/A',
               

Note: a path on the local filesystem is always denoted as "dir" (short for directory), instead of folder, path, or location. Especially location can be confusing in the context of geospatial modeling.

It is also possible to store and load custom configuration files. For more information, see [system setup](https://ewatercycle.readthedocs.io/en/latest/system_setup.html#configure-ewatercycle)

## Parameter sets

eWaterCycle allows model parameters to be set once a model instance has been created. However, as some models require a large number of files and other settings, it is also possible to pass a `ParameterSet` apon creation of a model instance. Which ParameterSets are available for a certain model can be requested from the model class.

In general the id determines the location in the collection of parameter sets available.

In [8]:
import ewatercycle.parameter_sets

In [9]:
ewatercycle.parameter_sets.available_parameter_sets(target_model='wflow')

('wflow_rhine_sbm_nc',)

In [12]:
parameter_set = ewatercycle.parameter_sets.get_parameter_set('wflow_rhine_sbm_nc')
print(parameter_set)

Parameter set
-------------
name='wflow_rhine_sbm_nc'
directory=PosixPath('/home/peter/ewatercycle/ewatercycle/docs/examples/parameter-sets/wflow_rhine_sbm_nc')
config=PosixPath('/home/peter/ewatercycle/ewatercycle/docs/examples/parameter-sets/wflow_rhine_sbm_nc/wflow_sbm_NC.ini')
doi='N/A'
target_model='wflow'
supported_model_versions={'2020.1.1'}


In [13]:
parameter_set.target_model

'wflow'

In [14]:
parameter_set.doi

'N/A'

In [15]:
parameter_set.name

'wflow_rhine_sbm_nc'

In [16]:
parameter_set.supported_model_versions

{'2020.1.1'}

In [18]:
print(parameter_set.directory)

/home/peter/ewatercycle/ewatercycle/docs/examples/parameter-sets/wflow_rhine_sbm_nc


## Forcing data

eWaterCycle can generate forcing for a model using the `forcing` module. This does all the required steps to go from the available datasets (ERA-5, ERA-Interim, etc) to whatever format the model requires for forcing. In general the output should match a forcing prepared "manually" for this model.

As input for the forcing generation the shape of the required forcing (usually a catchment) is needed. eWaterCycle will derive additiona information such as a bounding box if this is required by the model (e.g. in case of a gridded model), and additional options may be available or even required for some models.

The forcing module generates a ESMValTool recipe using all the arguments supplied. For most models all computations are done using ESMValTool. However, for some models (e.g. lisflood) additional computation is done, as some steps require data and/or code not available in ESMValTool.

The forcing generated is returned in the form of an object with metadata attached, and the main forcing being stored in a directory. This directory can either be specified explicitly, or it will be auto-generated by eWaterCycle from the output dir specified in the configuration. Some additional functionality is available to analyse the forcing, most promonently a plotting function to visually inspect the forcing.

#### ESMValTool configuration

As eWaterCycle relies on ESMValTool for processing forcing data, configuration for forcing is mostly defered to the esmvaltool configuration file. What ESMValTool configuration file to use can be specified in the eWatercycle configuration.

### Loading existing forcing data

Existing forcing can also be loaded using a function much like the forcing generation function

In [ ]:
import ewatercycle.forcing

#load an existing forcing that was generated manually by some scientist from Deltares
forcing = ewatercycle.forcing.load_external(
    #location on the file system where the output can be found
    directory='/path/to/forcing/output/',
    target_model='wflow', 
    start_time='2021-05-07T13:32:00Z'
    model_specific_forcing_info = {
        'forcing_filename' : 'some-filename.nc', # relative to forcing directory
        'temperature-variable-name': 'TEMP'
    },
)

#fetch an existing forcing that was generated with generate
forcing = ewatercycle.forcing.load(
    #location on the file system where the output can be found (no other options here)
    directory='/path/to/forcing/output/'
)

### Generating forcing data

In [2]:
import ewatercycle.forcing

forcing = ewatercycle.forcing.generate(
    target_model='wflow', 
    dataset='ERA-Interim', # example of a more advanced case: forcing.findCMIPData(mip=6, exp=historical)
    start_time="2021-05-07T13:32:00Z",
    end_time="2021-12-31T23:59:59Z",
    shape = '/path/to/shapefile.shp',
    model_specific_options = {
        'wflow_dem_file' : '/some/path/to/dem.dem',
        'hype_catchment_delineation' : '/some/other/shapefile'
    }
)

# Calling this function should show a progress bar, or at least auto-hide or auto-scroll the output.

/Users/bvreede/anaconda3/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/_warnings.py:18: UserWarning: 
  Thank you for trying out the new ESMValCore API.
  Note that this API is experimental and may be subject to change.
  More info: https://github.com/ESMValGroup/ESMValCore/issues/498
/Users/bvreede/anaconda3/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/config/_config_validators.py:254: ESMValToolDeprecationWarning: `write_plots` will be removed in 2.4.0.
/Users/bvreede/anaconda3/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/config/_config_validators.py:255: ESMValToolDeprecationWarning: `write_netcdf` will be removed in 2.4.0.


TypeError: generate() got an unexpected keyword argument 'directory'

In [ ]:
forcing.directory
# path to forcing output (content of the work dir of the esmvaltool output)

forcing.start_time
# datetime() object

forcing.end_time
# datetime() object

forcing.dataset
# 'ERA-Interim', eventually dataset object

forcing.target_model
# 'wflow'

forcing.shape
# Shape()

#optional
forcing.plot()
# some matplotlib output that shows the forcing

#optional
forcing.interactive_plot()
#some geoviews/widget/thingy that shows the forcing

#optional
forcing.log
# show some info on how this was generated (the esmvaltool log?)

#optional
forcing.recipe
# the esmvaltool recipe, to inspect what was done in the end.

### Example Yaml file with forcing metadata

```yaml
target_model: wflow 
start_time: 2021-05-07T13:32:00Z
end_time: 2021-12-31T23:59:59Z

#relative to (and inside) directory of this yaml file
shape: shapefile.shp
model_specific_forcing_info:
  # relative to this directory
  forcing_filename: some-filename.nc
  temperature-variable-name: TEMP
```

## Models

### Creating, setting up, and initializing a model instance

Now that we have created a forcing and selected a version of the model and a parameter set, we can create a model instance. When creating an instance we will have to select the version of the model to use, a parameter set, and a forcing. This will all be combined into an instance object. This can then be inspected for available parameters and default values, setup to create the configuration file, work_dir containing input files, and container, and initialized to prepare a model for running.

The way models are created, setup, and initialized matches PyMT as much as possible. There is currently no 'run' method as this convenance method makes it harder for users to create a more advanced usecase from a simple example.

### Model versions

eWaterCycle supports a number of models, and new once can be adding using a straightforward process. These are represented using a Python Class (e.g. ewatercycle.models.Wflow). Several version of a model may be available. To help with reproducibility the version of a model must always be specified when creating a model instance. A class function is available to retrieve a list of model versions.


In [ ]:
import ewatercycle.models.Wflow

ewatercycle.models.Wflow.available_versions
# ["2019.1", "2020.1"]

In [ ]:
import ewatercycle.models

#A parameter set and forcing object must be loaded or found before instanciating a model.

#Creates as model instance from a model version, parameter_set, and forcing
#version = mandatory
#parameter set = optional (e.g. MARRMOT does not need one)
#forcing = mandatory for now. At some point we may want to support feeding forcing to a model
#while running it using set_variable('temperature') and such.
model_instance = ewatercycle.models.Wflow(version='2019.1', parameter_set=parameter_set, forcing=forcing)

#using the parameters property all parameter defaults can be obtained.
#this is possibly a subset of everything that can be configured in the config file of model, and up to the
#creator of the model class to implement. For non-science settings such as logging settings, or file names
# for all parameter files it does not always make sense to expose these.
model_instance.parameters
#soil_depth: 9 (defaults to value in parameter set e.g. in a config template)
#start_time="2021-05-07T13:32:00Z" (defaults to start of forcing)
#end_time="2021-05-07T13:32:00Z" (defaults to end of forcing)
#not all parameter files can be set, but for some it may make sense.
#land_mask='some/land/mask/in/the/parameter_set.dem'

#the Setup function does the following:
#- Create a directory which serves as the current working directory for the mode instance
#- Creates a configuration file in this working directory based on the settings
#- Creates a container instance for the exact version of the model requesed
#- Makes the forcing, parameter set and and working directory available to the container using mounts.
#- If a model cannot cope with forcing and parameter set outside the working directory it is copied
#  to the working_directory instead.
#- Input is mounted read-only, the working directory is mounted read-write.
#- Setup will complain about incompatible model version, parameter_set, and forcing.
cfg_file, work_dir = model_instance.setup(
    land_mask='/some/land/mask.dem', # if outside of mounts, add a mount, or copy into working dir, or :'(
    soil_depth=9,
    start_time="2021-05-07T13:32:00Z",
    end_time="2021-05-07T13:32:00Z"
)

#After setup but before initialize everything is good-to-go, but nothing has been done yet. This is
#An opportunity to inspect the generated configuration file, and make any changes manually that could not be
#done through the setup method. Splitting these also makes it easier to run initialize in parallel in case a
#lot of models are created simultaneously (e.g. when calibrating a model)

# To modify, open it in an editor and save
print(cfg_file)

#This function will initialize the model using the files created above. For some models this can take some time.
model_instance.initialize(cfg_file)

### Running a model and getting output

Once initalized a model_instanced can be used by calling functions for running a single timestep (`update`), setting variables, and getting variables. Besides the rather lowlevel BMI functions several convenience functions are also available. These returns objects that make sense for the type of values returned, such as pandas DataFram and xarray DataArray or Dataset.

#### TODO Merge these cells below
In the loop, only use get_value_at_coords to keep track of a single outlet point.
At the end, show get_value_as_xarray to show you can also get the whole field

In [ ]:
#example storing all output of a certain field.
output = []
while model_instance.time < model_instance.end_time:
    
    # Update the model (takes a few seconds per timestep)
    model_instance.update() 
    
    #store entire discharge field
    discharge = model.get_value_as_xarray('Discharge')
    output.append(discharge)
        
    # Show progress
    print(reference.time, end="\r")  # "\r" clears the output before printing the next timestamp
    
result = xarray.merge(output)

result
#xarray with full output of discharge field

In [ ]:
#example storing a timeseries for a single location of a certain field

#some location of interest within the model
output_latitude = 55.4
output_longitude = 20.0

simulated_discharge = pd.DataFrame(index='time', columns=['discharge'])
while model_instance.time < model_instance.end_time:
    
    # Update the model (takes a few seconds per timestep)
    model_instance.update() 
    
    # Track discharge at station location
    discharge = model_instance.get_value_at_location('Discharge', latitude=output_latitude, longitude=output_longitude, method='nearest')
    simulated_discharge.append({"time":reference.time, "discharge": discharge})
    
    # Show progress
    print(model_instance.time, end="\r")  # "\r" clears the output before printing the next timestamp
    
model_output
#nice table of all dischage values over time for a single location

## Observations

In [ ]:
#Read GRDC data

import ewatercycle.observation.grdc

grdc_station_id = 4147380

#This function automatically fetches the location of the GRDC data from the configuration file
#Start and end dates are fetched from the model instance
observations = ewatercycle.observation.grdc.get_grdc_data(
    station_id=grdc_station_id,
    start_date=model_instance.start_time.date(),
    end_date=model_instance.end_time.date(),
)
observations
#xarray containing grdc data

In [ ]:
#Combine simulated and observated discharge into a single dataframe

import pandas
 
simulated_discharge_df = pandas.DataFrame(
    {'simulation': model_output}, index=timestamps
)
observations_df = observations.streamflow.to_dataframe().rename(
    columns={'streamflow': 'observation'}
)
discharge = simulated_discharge_df.join(observations_df)
discharge

#table with simulated and observed discharge

## Analysis
Once a model has run we can analyse the result. For this example we will assume a DataFrame was created with values over time for a certain location for which GRDC station data is available

In [ ]:
#Plot hydrograph

import ewatercycle.analysis

#todo: also add forcing to this hydrograph in some smart way
ewatercycle.analysis.hydrograph(
    discharge=discharge,
    reference='observation',
)

#nice hydrograph

In [ ]:
#finalize the model.
#as a side effect also destroys the container
model_instance.finalize()